In [7]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

# Load model from Hugging Face
model_name = "model/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

# Create a concrete function for conversion
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

@tf.function
def model_fn(input_ids, attention_mask):
    outputs = model([input_ids, attention_mask])
    return outputs.last_hidden_state[:, 0, :]

# Convert function to concrete function
concrete_func = model_fn.get_concrete_function(
    input_ids=tf.TensorSpec([1, 128], tf.int32),
    attention_mask=tf.TensorSpec([1, 128], tf.int32)
)

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

# Save the model
with open("minilm_l6_v2.tflite", "wb") as f:
    f.write(tflite_model)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
2025-03-20 00:52:06.651084: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
